# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [15]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [59]:
data=pd.read_csv("census_data.csv")

In [62]:
data.tail()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [6]:
data['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [63]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [64]:
data['income_bracket']=data['income_bracket'].apply(label_fix)
#census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [65]:
x_data = data.drop('income_bracket',axis=1)
y_labels = data['income_bracket']


### Perform a Train Test Split on the Data

In [66]:
x_train, x_test, Y_train, Y_test = train_test_split(x_data, y_labels, test_size=0.3, random_state=101)

In [67]:
#

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [68]:
data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [69]:
#

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [70]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [71]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [72]:
feat_list=[gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [73]:
input_fn=tf.estimator.inputs.pandas_input_fn(x=x_train,y=Y_train,batch_size=10,num_epochs=None,shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [74]:
model = tf.estimator.LinearClassifier(feature_columns=feat_list)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmp4xwjkq9u', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x122b974a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [75]:
model.train(input_fn, steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmp4xwjkq9u/model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 189.782
INFO:tensorflow:loss = 4.429467, step = 101 (0.531 sec)
INFO:tensorflow:global_step/sec: 402.143
INFO:tensorflow:loss = 57.560986, step = 201 (0.246 sec)
INFO:tensorflow:global_step/sec: 396.723
INFO:tensorflow:loss = 4.5567513, step = 301 (0.252 sec)
INFO:tensorflow:global_step/sec: 390.59
INFO:tensorflow:loss = 1.8060349, step = 401 (0.257 sec)
INFO:tensorflow:global_step/sec: 398.28
INFO:tensorflow:loss = 3.2214663, step = 501 (0.249 sec)
INFO:tensorflow:global_step/sec: 391.383
INFO:tensorflow:loss = 4.193394, step = 601 (0.256 sec)
INFO:tensorflow:gl

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [76]:
eval_fn=tf.estimator.inputs.pandas_input_fn(x=x_test, batch_size=len(x_test), num_epochs=1,shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [77]:
predictions = model.predict(input_fn=eval_fn)
lpred=list(predictions)
#results = model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmp4xwjkq9u/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [78]:
lpred[0]

{'logits': array([-1.4367727], dtype=float32),
 'logistic': array([0.19204561], dtype=float32),
 'probabilities': array([0.8079544 , 0.19204561], dtype=float32),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [79]:
final_preds = []
for pred in lpred:
    final_preds.append(pred['class_ids'][0])

In [80]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [81]:
from sklearn.metrics import classification_report

In [82]:
print(classification_report(Y_test,final_preds))

             precision    recall  f1-score   support

          0       0.88      0.91      0.89      7436
          1       0.68      0.59      0.63      2333

avg / total       0.83      0.84      0.83      9769



In [83]:
#DNN

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpwje_gpab', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x123065b70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [96]:
embedded_group_column_gender = tf.feature_column.embedding_column(gender, dimension=2)
embedded_group_column_occuption = tf.feature_column.embedding_column(occupation, dimension=15)
embedded_group_column_ms = tf.feature_column.embedding_column(marital_status, dimension=7)
embedded_group_column_relation = tf.feature_column.embedding_column(relationship, dimension=6)
embedded_group_column_edu = tf.feature_column.embedding_column(education, dimension=16)
embedded_group_column_workc = tf.feature_column.embedding_column(workclass, dimension=9)
embedded_group_column_nativec = tf.feature_column.embedding_column(native_country, dimension=42)

In [94]:
data['native_country'].unique()

array([' United-States', ' Cuba', ' Jamaica', ' India', ' ?', ' Mexico',
       ' South', ' Puerto-Rico', ' Honduras', ' England', ' Canada',
       ' Germany', ' Iran', ' Philippines', ' Italy', ' Poland',
       ' Columbia', ' Cambodia', ' Thailand', ' Ecuador', ' Laos',
       ' Taiwan', ' Haiti', ' Portugal', ' Dominican-Republic',
       ' El-Salvador', ' France', ' Guatemala', ' China', ' Japan',
       ' Yugoslavia', ' Peru', ' Outlying-US(Guam-USVI-etc)', ' Scotland',
       ' Trinadad&Tobago', ' Greece', ' Nicaragua', ' Vietnam', ' Hong',
       ' Ireland', ' Hungary', ' Holand-Netherlands'], dtype=object)

In [97]:
feat_list=[embedded_group_column_gender,embedded_group_column_occuption,embedded_group_column_ms,embedded_group_column_relation,
           embedded_group_column_edu,embedded_group_column_workc,embedded_group_column_nativec,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [98]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,y=Y_train,batch_size=10,num_epochs=None,shuffle=True)

In [99]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_list,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpvan_sspu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x122bac320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [100]:
dnn_model.train(input_fn=input_func,steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpvan_sspu/model.ckpt.
INFO:tensorflow:loss = 391.59756, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpvan_sspu/model.ckpt.
INFO:tensorflow:Loss for final step: 6.100886.


In [101]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=x_test,
      y=Y_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [102]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-07-23:15:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpvan_sspu/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-07-23:15:31
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.7854437, accuracy_baseline = 0.7611833, auc = 0.6394565, auc_precision_recall = 0.46747348, average_loss = 0.9464054, global_step = 100, label/mean = 0.23881666, loss = 9.463085, precision = 0.6453988, prediction/mean = 0.2821161, recall = 0.22546078


{'accuracy': 0.7854437,
 'accuracy_baseline': 0.7611833,
 'auc': 0.6394565,
 'auc_precision_recall': 0.46747348,
 'average_loss': 0.9464054,
 'label/mean': 0.23881666,
 'loss': 9.463085,
 'precision': 0.6453988,
 'prediction/mean': 0.2821161,
 'recall': 0.22546078,
 'global_step': 100}

In [103]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test),shuffle=False)

In [105]:
predictions=list(dnn_model.predict(pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/c3/88hk3vsx7vvbx2k7w_c2wwtshd1228/T/tmpvan_sspu/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [106]:
predictions[0]

{'logits': array([-1.4742498], dtype=float32),
 'logistic': array([0.18629752], dtype=float32),
 'probabilities': array([0.8137025 , 0.18629752], dtype=float32),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object)}

In [107]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [108]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [110]:
print(classification_report(Y_test,final_preds))

             precision    recall  f1-score   support

          0       0.80      0.96      0.87      7436
          1       0.65      0.23      0.33      2333

avg / total       0.76      0.79      0.74      9769



In [ ]:
'
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)
'

# Great Job!